In [20]:
import cv2
import tkinter as tk
import tkinter.filedialog as tkfd
import os
import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import  Flatten, Dense, Activation,Convolution2D,MaxPooling2D
from sklearn.cross_validation import train_test_split
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from keras import backend as K
import random
%matplotlib inline

In [3]:
def load_images(folder):
        image_list=[]
        for filename in os.listdir(folder):
            img=cv2.imread(os.path.join(folder,filename))
            if img is not None:
                image_list.append(img)
        return image_list

In [4]:
tk.Tk().withdraw()
path_Train=tkfd.askdirectory()

In [5]:
X_Train=load_images(path_Train)
X_Train=np.array(X_Train)

In [6]:
tk.Tk().withdraw()
path_Test=tkfd.askdirectory()

In [7]:
X_Test=load_images(path_Test)
X_Test=np.array(X_Test)

In [8]:
tk.Tk().withdraw()
path_Validation=tkfd.askdirectory()

In [9]:
X_Validation=load_images(path_Validation)
X_Validation=np.array(X_Validation)

In [10]:
os.chdir(path=tkfd.askdirectory())
data_Train=pd.read_csv("ISIC-2017_Training_Part3_GroundTruth.csv")
data_Test=pd.read_csv("ISIC-2017_Test_v2_Part3_GroundTruth.csv")
data_Validation=pd.read_csv("ISIC-2017_Validation_Part3_GroundTruth.csv")

In [11]:
data_Train=data_Train.iloc[0:2000,1]
data_Test=data_Test.iloc[0:600,1]
data_Validation=data_Validation.iloc[0:300,1]

y_Train=data_Train
y_Test=data_Test
y_Validation=data_Validation

y_Train=np.array(y_Train)
y_Test=np.array(y_Test)
y_Validation=np.array(y_Validation)

In [12]:
y_Train = np_utils.to_categorical(data_Train)
y_Test = np_utils.to_categorical(data_Test)
y_Validation = np_utils.to_categorical(data_Validation)

In [13]:
# Convolutional Neural Network with 6 layers
Optimizer=Adam(lr=0.001)
objective='binary_crossentropy'
def center_normalize(x):
    return (x-K.mean(x))/K.std(x)
model=Sequential()
#input layer
model.add(Activation(activation=center_normalize, input_shape=(64, 64,3)))
# convolutional layer
model.add(Convolution2D(32,5,5,border_mode='same',activation='relu',dim_ordering='tf'))
#pooling layer
model.add(MaxPooling2D(pool_size=(2,2),dim_ordering='tf'))
# convolutional layer
model.add(Convolution2D(64,3,3,border_mode='same',activation='relu',dim_ordering='tf'))
# pooling layer
model.add(MaxPooling2D(pool_size=(2,2),dim_ordering='tf'))
model.add(Flatten())
# Relu 
model.add(Dense(128, activation='relu'))
# Sigmoid Fully connected layer
model.add(Dense(y_Train.shape[1]))
model.add(Activation('sigmoid'))
model.compile(loss=objective,optimizer=Optimizer,metrics=['accuracy'])

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (5, 5), activation="relu", padding="same", data_format="channels_last")`
  # Remove the CWD from sys.path while we load stuff.
C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(pool_size=(2, 2), data_format="channels_last")`
  if sys.path[0] == '':
C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu", padding="same", data_format="channels_last")`
  
C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(pool_size=(2, 2), data_format="channels_last")`
  app.launch_new_instance()


In [29]:
random.seed(250)
model.fit(X_Train,y_Train,batch_size=200,nb_epoch=15,verbose=1)

C:\Users\User\Anaconda3\lib\site-packages\keras\models.py:942: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/15
2000/2000 [==============================] - 20s 10ms/step - loss: 0.4620 - acc: 0.8132
Epoch 2/15
2000/2000 [==============================] - 18s 9ms/step - loss: 0.4617 - acc: 0.8152
Epoch 3/15
2000/2000 [==============================] - 20s 10ms/step - loss: 0.4652 - acc: 0.8135
Epoch 4/15
2000/2000 [==============================] - 18s 9ms/step - loss: 0.4603 - acc: 0.8137
Epoch 5/15
2000/2000 [==============================] - 18s 9ms/step - loss: 0.4585 - acc: 0.8150
Epoch 6/15
2000/2000 [==============================] - 18s 9ms/step - loss: 0.4491 - acc: 0.8140
Epoch 7/15
2000/2000 [==============================] - 18s 9ms/step - loss: 0.4467 - acc: 0.8162
Epoch 8/15
2000/2000 [==============================] - 18s 9ms/step - loss: 0.4451 - acc: 0.8180
Epoch 9/15
2000/2000 [==============================] - 18s 9ms/step - loss: 0.4445 - acc: 0.8178
Epoch 10/15
2000/2000 [==============================] - 18s 9ms/step - loss: 0.4349 - acc: 0.8175
Epoch 11/15
2000/

In [30]:
scores = model.evaluate(X_Test, y_Test)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

600/600 [==============================] - 2s 4ms/step

acc: 80.25%


In [31]:
scores = model.evaluate(X_Validation, y_Validation)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

150/150 [==============================] - 1s 4ms/step

acc: 78.67%
